In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Setup

In [2]:
import pandas as pd

import pickle
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from os import path
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
%ls gdrive/MyDrive/CodiEsp/

background/  dev/  README.txt  test/  train/


In [4]:
df_train = pd.read_csv('drive/MyDrive/CodiEsp/train/trainD.tsv', sep='\t', header=None)
df_train.rename(columns={0:"Id", 1:"ICD10"}, inplace=True)
print("Training Data:")
display(df_train.head())

print("\n\nValidation Data:")
df_val = pd.read_csv('drive/MyDrive/CodiEsp/dev/devD.tsv', sep='\t', header=None)
df_val.rename(columns = {0:"Id", 1:"ICD10"}, inplace = True)
display(df_val.head())

print("\n\nTest Data:")
df_test = pd.read_csv('drive/MyDrive/CodiEsp/test/testD.tsv', sep='\t', header=None)
df_test.rename(columns={0:"Id", 1:"ICD10"}, inplace=True)
display(df_test.head())

df = pd.concat([df_train, df_val, df_test])

Training Data:


,Id,ICD10
0,S0004-06142005000700014-1,n44.8
1,S0004-06142005000700014-1,z20.818
2,S0004-06142005000700014-1,r60.9
3,S0004-06142005000700014-1,r52
4,S0004-06142005000700014-1,a23.9




Validation Data:


,Id,ICD10
0,S0004-06142005000900016-1,q62.11
1,S0004-06142005000900016-1,n28.89
2,S0004-06142005000900016-1,n39.0
3,S0004-06142005000900016-1,r31.9
4,S0004-06142005000900016-1,n23




Test Data:


,Id,ICD10
0,S0004-06142005000500011-1,s22.49xa
1,S0004-06142005000500011-1,n28.1
2,S0004-06142005000500011-1,r69
3,S0004-06142005000500011-1,f17.210
4,S0004-06142005000500011-1,r31.9


In [5]:
ids = df['Id'].unique()
codes = df['ICD10'].unique()  

print("Number of documents in training data:", len(ids), "\nNumber of ICD10 codes:", len(codes))

Number of documents in training data: 1000 
Number of ICD10 codes: 2557


In [6]:
code2idx = {}
for i in range(len(codes)):
  code2idx[codes[i]] = i

id2label = {}
for i in range(len(ids)):
  id2label[ids[i]] = [0]*len(codes)

for i, data in df.iterrows():
  _id = data[0]
  _code = data[1]
  id2label[_id][code2idx[_code]] = 1

_id2label = [(id, y) for id, y in id2label.items()]
ID, Y = zip(*_id2label)

In [7]:
stop_words = stopwords.words('english')

In [8]:
def remstopwords(text, stopwords):
    text = re.sub('\[\*\*[^\]]*\*\*\]', '', text)
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', ' ', text.lower()) 
    text = re.sub(" \d+", " ", text)
    return " ".join([i for i in text.split() if i not in stopwords])

In [9]:
X_train = []
Y_train = []

for id in (df_train['Id'].unique()):
  Y_train.append(id2label[id])

  # with open('gdrive/MyDrive/CodiEsp/train/text_files_en/' + id + '.txt', 'r') as f:
  #   text = f.read().replace('\n', ' ')
  # X_train.append(remstopwords(text.lower(), stop_words))
# with open("gdrive/MyDrive/X_train.txt", "wb") as fp:
#   pickle.dump(X_train, fp)
with open("drive/MyDrive/X_train.txt", "rb") as fp:
  X_train = pickle.load(fp)

In [10]:
X_val = []
Y_val = []

for id in (df_val['Id'].unique()):
  Y_val.append(id2label[id])

  # with open('gdrive/MyDrive/CodiEsp/dev/text_files_en/' + id + '.txt', 'r') as f:
  #   text = f.read().replace('\n', ' ')
  # X_val.append(remstopwords(text.lower(), stop_words))
# with open("gdrive/MyDrive/X_val.txt", "wb") as fp:
#   pickle.dump(X_val, fp)
with open("drive/MyDrive/X_val.txt", "rb") as fp:
  X_val = pickle.load(fp)

In [11]:
X_test = []
Y_test = []

for id in (df_test['Id'].unique()):
  Y_test.append(id2label[id])

#   with open('gdrive/MyDrive/CodiEsp/test/text_files_en/' + id + '.txt', 'r') as f:
#     text = f.read().replace('\n', ' ')
#   X_test.append(remstopwords(text.lower(), stop_words))
# with open("gdrive/MyDrive/X_test.txt", "wb") as fp:
#   pickle.dump(X_val, fp)
with open("drive/MyDrive/X_test.txt", "rb") as fp:
  X_test = pickle.load(fp)

In [16]:
X_train[0]

'describe case year old man previous active life complained osteoarticular pain variable location last month fever last week peaks morning evening c onset symptoms extremadura endemic region brucella taking pasteurized milk cheese cattle among diners appeared several cases brucellosis admission study febrile syndrome epidemiological history possible exposure brucella presents picture right orchiepididymitis physical examination revealed ta c mmhg fc bpm conscious oriented sweating eupneic good nutritional status hydration adenopathy goiter jugular vein engorgement symmetrical carotid pulses present head neck auscultation cardiac arrhythmia murmurs ropes extracts pulmonary emphysema vesicular murmur conservation abdomen blade depressible without masses enlargement neurological examination revealed meningeal signs focality data extremities without varices edemas peripheral peace present symmetrical urological examination revealed enlarged right test attached skin fluctuating areas intens

In [12]:
p_code = [0]*len(codes)
for label in Y_train:
  for i, code in enumerate(label):
    if (code == 1):
      p_code[i] = 1

not_present = 0
for i, present in enumerate(p_code):
  if (present == 0):
    not_present += 1

print("Number of classes NOT PRESENT in training dataset:", not_present)

Number of classes NOT PRESENT in training dataset: 790


In [13]:
def hamming_score(y_true, y_pred, normalize = True, sample_weight = None):
    ''' Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
        http://stackoverflow.com/q/32239577/395857 '''

    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set(np.where(y_true[i])[0])
        set_pred = set(np.where(y_pred[i])[0])
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float(len(set_true.union(set_pred)))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

def print_score(y_pred, y_t, clf):
    print("Clf: ", clf.__class__.__name__)
    print("Hamming loss: {}".format(hamming_loss(y_pred, y_t)))
    print("Hamming score: {}".format(hamming_score(y_pred, y_t)))
    print("---")    

In [14]:
SOS = 0
EOS = 1
PAD = 2
UNK = 3

dictionary = {'START' : (1, SOS), 'END' : (1, EOS), 'PAD' : (1, PAD), 'UNK': (1, UNK)}

max_length = -1
index = 4
for i in range(len(X_train)):
  if i%50 == 0:
    print(f'[INFO] At {i}th line, making the dictionary...')
  tokenized = nltk.word_tokenize(X_train[i])
  if len(tokenized) > max_length:
    max_length = len(tokenized)
  for t in tokenized:
    if t not in dictionary.keys():
      dictionary[t] = (1, index)
      index += 1
    else:
      dictionary[t] = (dictionary[t][0]+1, dictionary[t][1])

for i in range(len(X_val)):
  if i%50 == 0:
    print(f'[INFO] At {i}th line, making the dictionary...')
  tokenized = nltk.word_tokenize(X_val[i])
  if len(tokenized) > max_length:
    max_length = len(tokenized)
  for t in tokenized:
    if t not in dictionary.keys():
      dictionary[t] = (1, index)
      index += 1
    else:
      dictionary[t] = (dictionary[t][0]+1, dictionary[t][1])

max_length = max_length + 1
print('-------------------------------------------------------')

print(f'[INFO] Maximum length of the documents is : {max_length}')
print(f'[INFO] Number of words in the dictionary : {len(dictionary)}')

[INFO] At 0th line, making the dictionary...
[INFO] At 50th line, making the dictionary...
[INFO] At 100th line, making the dictionary...
[INFO] At 150th line, making the dictionary...
[INFO] At 200th line, making the dictionary...
[INFO] At 250th line, making the dictionary...
[INFO] At 300th line, making the dictionary...
[INFO] At 350th line, making the dictionary...
[INFO] At 400th line, making the dictionary...
[INFO] At 450th line, making the dictionary...
[INFO] At 0th line, making the dictionary...
[INFO] At 50th line, making the dictionary...
[INFO] At 100th line, making the dictionary...
[INFO] At 150th line, making the dictionary...
[INFO] At 200th line, making the dictionary...
-------------------------------------------------------
[INFO] Maximum length of the documents is : 563
[INFO] Number of words in the dictionary : 13670


In [15]:
X_train_indexed = []
X_val_indexed = []
X_test_indexed = []

for i in range(len(X_train)):
  tokenized = nltk.word_tokenize(X_train[i])
  tokenized = list(map(lambda x: dictionary[x][1], tokenized))
  tokenized.append(dictionary['END'][1])
  X_train_indexed.append(tokenized)

for i in range(len(X_val)):
  tokenized = nltk.word_tokenize(X_val[i])
  tokenized = list(map(lambda x: dictionary[x][1], tokenized))
  tokenized.append(dictionary['END'][1])
  X_val_indexed.append(tokenized)

for i in range(len(X_test)):
  tokenized = nltk.word_tokenize(X_test[i])
  tokenized = list(map(lambda x: dictionary[x][1], tokenized))
  tokenized.append(dictionary['END'][1])
  X_test_indexed.append(tokenized)

In [16]:
X_train_indexed = tf.keras.preprocessing.sequence.pad_sequences(X_train_indexed, maxlen=max_length, padding='post', value=dictionary['PAD'][1], dtype='float32')
X_val_indexed = tf.keras.preprocessing.sequence.pad_sequences(X_val_indexed, maxlen=max_length, padding='post', value=dictionary['PAD'][1], dtype='float32')
X_test_final = tf.keras.preprocessing.sequence.pad_sequences(X_test_indexed, maxlen=max_length, padding='post', value=dictionary['PAD'][1], dtype='float32')
Y_train_indexed = np.array(Y_train)
Y_val_indexed = np.array(Y_val)
Y_test_indexed = np.array(Y_test)

In [17]:
from sklearn.metrics import f1_score

# Model 1

In [ ]:
model_1 = tf.keras.models.Sequential([
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(512, activation='relu', input_shape=(max_length,)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(Y_train_indexed.shape[1], activation='sigmoid')
])

In [ ]:
model_1.compile(loss = 'binary_crossentropy' , optimizer = 'rmsprop' , metrics = ['accuracy'])

In [ ]:
model_1.fit(X_train_indexed, Y_train_indexed, epochs = 100, batch_size = 64)

Epoch 1/100
8/8 [==============================] - 2s 4ms/step - loss: 0.3690 - accuracy: 0.0149 
Epoch 2/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0259 - accuracy: 0.0839
Epoch 3/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0253 - accuracy: 0.1112
Epoch 4/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0248 - accuracy: 0.1244
Epoch 5/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0248 - accuracy: 0.0846
Epoch 6/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0240 - accuracy: 0.0774
Epoch 7/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0239 - accuracy: 0.1365
Epoch 8/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0250 - accuracy: 0.1318
Epoch 9/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0225 - accuracy: 0.1582
Epoch 10/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0226 - accuracy: 0.1552
Epoch 11/100
8/8 [==========

In [ ]:
predictions_train_1 = model_1.predict(X_train_indexed)
predictions_rounded_train_1 = np.zeros(predictions_train_1.shape)

for i in range(len(predictions_train_1)):
  predictions_rounded_train_1[i] = np.where(predictions_train_1[i] < 0.2, 0, 1)

hamming_score(Y_train_indexed, predictions_rounded_train_1)

0.9764100688679406

In [ ]:
predictions_val_1 = model_1.predict(X_val_indexed)
predictions_rounded_val_1 = np.zeros(predictions_val_1.shape)

for i in range(len(predictions_val_1)):
  predictions_rounded_val_1[i] = np.where(predictions_val_1[i] < 0.2, 0, 1)

hamming_score(Y_val_indexed, predictions_rounded_val_1)

0.0188622702501768

In [ ]:
f1_score(Y_val_indexed, predictions_rounded_val_1, average='macro')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.0018383386351728862

In [ ]:
model_1.save('gdrive/MyDrive/model_1.hdf5')

# Model 2

In [ ]:
model_2 = tf.keras.models.Sequential([
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation='relu', input_shape = (max_length,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(Y_train_indexed.shape[1], activation='sigmoid')
])

In [ ]:
model_2.compile(loss='binary_crossentropy' , optimizer='rmsprop' , metrics=['accuracy'])

In [ ]:
model_2.fit(X_train_indexed, Y_train_indexed, epochs = 30, batch_size = 64)

Epoch 1/30
8/8 [==============================] - 2s 6ms/step - loss: 0.3779 - accuracy: 0.0209 
Epoch 2/30
8/8 [==============================] - 0s 6ms/step - loss: 0.0245 - accuracy: 0.1094
Epoch 3/30
8/8 [==============================] - 0s 6ms/step - loss: 0.0232 - accuracy: 0.1248
Epoch 4/30
8/8 [==============================] - 0s 6ms/step - loss: 0.0231 - accuracy: 0.1216
Epoch 5/30
8/8 [==============================] - 0s 6ms/step - loss: 0.0227 - accuracy: 0.1159
Epoch 6/30
8/8 [==============================] - 0s 6ms/step - loss: 0.0219 - accuracy: 0.1357
Epoch 7/30
8/8 [==============================] - 0s 6ms/step - loss: 0.0212 - accuracy: 0.1370
Epoch 8/30
8/8 [==============================] - 0s 6ms/step - loss: 0.0209 - accuracy: 0.1585
Epoch 9/30
8/8 [==============================] - 0s 6ms/step - loss: 0.0193 - accuracy: 0.1565
Epoch 10/30
8/8 [==============================] - 0s 6ms/step - loss: 0.0185 - accuracy: 0.1735
Epoch 11/30
8/8 [=====================

In [ ]:
predictions_train_2 = model_2.predict(X_train_indexed)
predictions_rounded_train_2 = np.zeros(predictions_train_2.shape)

for i in range(len(predictions_train_2)):
  predictions_rounded_train_2[i] = np.where(predictions_train_2[i] < 0.5, 0, 1)

hamming_score(Y_train_indexed, predictions_rounded_train_2)

0.7522703384761157

In [ ]:
predictions_val_2 = model_2.predict(X_val_indexed)
predictions_rounded_val_2 = np.zeros(predictions_val_2.shape)

for i in range(len(predictions_val_2)):
  predictions_rounded_val_2[i] = np.where(predictions_val_2[i] < 0.5, 0, 1)

hamming_score(Y_val_indexed, predictions_rounded_val_2)

0.006815762771207418

In [ ]:
f1_score(Y_val_indexed, predictions_rounded_val_2, average='macro')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.000762746941440898

In [ ]:
model_2.save('drive/MyDrive/model_2.hdf5')

# Model 3

In [50]:
model_3 = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(len(dictionary), 1024, input_length = max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dense(Y_train_indexed.shape[1], activation='sigmoid')
])

In [51]:
model_3.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 563, 1024)         13998080  
_________________________________________________________________
bidirectional_9 (Bidirection (None, 563, 256)          1180672   
_________________________________________________________________
dropout_13 (Dropout)         (None, 563, 256)          0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 256)               394240    
_________________________________________________________________
dropout_14 (Dropout)         (None, 256)               0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 256)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)              

In [52]:
# early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, verbose=0, restore_best_weights=True)
optimizer = tf.keras.optimizers.Adam(lr=0.002, clipnorm=1)

In [53]:
model_3.compile(loss='binary_crossentropy' , optimizer=optimizer , metrics=['accuracy'])

In [54]:
model_3.fit(X_train_indexed, Y_train_indexed, epochs=40, batch_size=64, validation_data=(X_val_indexed, Y_val_indexed,))

Epoch 1/40
8/8 [==============================] - 16s 1s/step - loss: 0.5619 - accuracy: 0.0000e+00 - val_loss: 0.3044 - val_accuracy: 0.0000e+00
Epoch 2/40
8/8 [==============================] - 7s 848ms/step - loss: 0.1339 - accuracy: 0.0283 - val_loss: 0.0879 - val_accuracy: 0.0400
Epoch 3/40
8/8 [==============================] - 7s 847ms/step - loss: 0.0403 - accuracy: 0.1234 - val_loss: 0.0413 - val_accuracy: 0.1120
Epoch 4/40
8/8 [==============================] - 7s 849ms/step - loss: 0.0271 - accuracy: 0.1399 - val_loss: 0.0319 - val_accuracy: 0.1120
Epoch 5/40
8/8 [==============================] - 7s 847ms/step - loss: 0.0263 - accuracy: 0.1283 - val_loss: 0.0302 - val_accuracy: 0.1120
Epoch 6/40
8/8 [==============================] - 7s 851ms/step - loss: 0.0251 - accuracy: 0.1336 - val_loss: 0.0319 - val_accuracy: 0.0400
Epoch 7/40
8/8 [==============================] - 7s 850ms/step - loss: 0.0256 - accuracy: 0.1334 - val_loss: 0.0356 - val_accuracy: 0.1120
Epoch 8/40
8/8

In [55]:
predictions_val_3 = model_3.predict(X_val_indexed)
predictions_rounded_val_3 = np.zeros(predictions_val_3.shape)

for i in range(len(predictions_val_3)):
  predictions_rounded_val_3[i] = np.where(predictions_val_3[i] < 0.2, 0, 1)

hamming_score(Y_val_indexed, predictions_rounded_val_3)

0.027089222093152183

In [49]:
f1_score(Y_val_indexed, predictions_rounded_val_3, average='macro')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.00397464075865705

In [77]:
model_3.save('drive/MyDrive/model_3.hdf5')

## Final Model

In [56]:
model_3 = tf.keras.models.load_model('drive/MyDrive/model_3.hdf5')

In [57]:
model_3.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 563, 512)          6999040   
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 563, 1024)         4198400   
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 512)               2623488   
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 512)               0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 512)               2048      
_________________________________________________________________
dense_10 (Dense)             (None, 256)              

In [58]:
predictions_val_3 = model_3.predict(X_val_indexed)
predictions_rounded_val_3 = np.zeros(predictions_val_3.shape)

for i in range(len(predictions_val_3)):
  predictions_rounded_val_3[i] = np.where(predictions_val_3[i] < 0.2, 0, 1)

hamming_score(Y_val_indexed, predictions_rounded_val_3)

0.05898184286120076

In [59]:
f1_score(Y_val_indexed, predictions_rounded_val_3, average='macro')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.0008161220077547298